# Chapter 8: Experimental design - the basics

## Libraries and data

### Libraries

In [3]:
# Common libraries
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import seaborn as sns

# Chapter-specific libraries
import statsmodels.stats.proportion as ssprop # To calculate the standardized effect size
import statsmodels.stats.power as ssp #To calculate the standard power

### Data

In [4]:
hist_data_df = pd.read_csv('chap8-historical_data.csv')
exp_data_df = pd.read_csv('chap8-experimental_data.csv')

## Determining random assignment and sample size/power

In [3]:
### Basic random assignment
K = 2
assgnt = np.random.uniform(0,1,1)
group = "control" if assgnt <= 1/K else "treatment"

In [4]:
effect_size = ssprop.proportion_effectsize(0.194, 0.184)
ssp.tt_ind_solve_power(effect_size = effect_size, 
                       alpha = 0.05, 
                       nobs1 = None, 
                       alternative = 'larger', 
                       power=0.8)

18950.818821558503

In [15]:
### Null experimental dataset
exp_null_data_df = hist_data_df.copy().sample(2000)
exp_null_data_df['oneclick'] = np.where(np.random.uniform(0,1,2000)>0.5, 1, 0)
mod = smf.logit('booked ~ oneclick + age + gender', data = exp_null_data_df)
mod.fit(disp=0).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 booked   No. Observations:                 2000
Model:                          Logit   Df Residuals:                     1996
Method:                           MLE   Df Model:                            3
Date:                Sat, 20 Mar 2021   Pseudo R-squ.:                  0.2588
Time:                        07:46:52   Log-Likelihood:                -696.57
converged:                       True   LL-Null:                       -939.75
Covariance Type:            nonrobust   LLR p-value:                4.318e-105
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          9.5764      0.621     15.412      0.000       8.359      10.794
gender[T.male]     0.1589      0.136      1.167      0.243      -0.108       0.426
oneclick           0.0496      0.136      0.365      0.715      -0.217       0.316
age               -0.3017      0.017    -17.434      0.000      -0.336      -0.268
==================================================================================
"""

In [8]:
### Function definitions

## Metric function
def log_reg_fun(dat_df):
    
    model = smf.logit('booked ~ oneclick + age + gender', data = dat_df)
    res = model.fit(disp=0)
    coeff = res.params['oneclick']
    return coeff

## Bootstrap CI function
def boot_CI_fun(dat_df, metric_fun, B = 100, conf_level = 0.9):
  #Setting sample size
  N = len(dat_df)
  conf_level = conf_level
  coeffs = []
  
  for i in range(B):
      sim_data_df = dat_df.sample(n=N, replace = True)
      coeff = metric_fun(sim_data_df)
      coeffs.append(coeff)
  
  coeffs.sort()
  start_idx = round(B * (1 - conf_level) / 2)
  end_idx = - round(B * (1 - conf_level) / 2)
  confint = [coeffs[start_idx], coeffs[end_idx]]  
  return(confint)

## decision function
def decision_fun(dat_df, metric_fun, B = 100, conf_level = 0.9):
    boot_CI = boot_CI_fun(dat_df, metric_fun, B = B, conf_level = conf_level)
    decision = 1 if boot_CI[0] > 0  else 0
    return decision 

## Function for single simulation
def single_sim_fun(Nexp, dat_df = hist_data_df, metric_fun = log_reg_fun, 
                   eff_size = 0.01, B = 100, conf_level = 0.9):
    
    #Adding predicted probability of booking
    hist_model = smf.logit('booked ~ age + gender + period', data = dat_df)
    res = hist_model.fit(disp=0)
    sim_data_df = dat_df.copy()
    sim_data_df['pred_prob_bkg'] = res.predict()
    #Filtering down to desired sample size
    sim_data_df = sim_data_df.sample(Nexp)
    #Random assignment of experimental groups
    sim_data_df['oneclick'] = np.where(np.random.uniform(size=Nexp) <= 0.5, 0, 1)
    # Adding effect to treatment group
    sim_data_df['pred_prob_bkg'] = np.where(sim_data_df.oneclick == 1, 
                                            sim_data_df.pred_prob_bkg + eff_size, 
                                            sim_data_df.pred_prob_bkg)
    sim_data_df['booked'] = np.where(sim_data_df.pred_prob_bkg >= \
                                     np.random.uniform(size=Nexp), 1, 0)
    
    #Calculate the decision (we want it to be 1)
    decision = decision_fun(sim_data_df, metric_fun = metric_fun, B = B, 
                            conf_level = conf_level)
     
    return decision  
 
## power simulation function
def power_sim_fun(dat_df, metric_fun, Nexp, eff_size, Nsim, B = 100, 
                  conf_level = 0.9):
    power_lst = []
    for i in range(Nsim):
        power_lst.append(single_sim_fun(Nexp = Nexp, dat_df = dat_df, 
                                        metric_fun = metric_fun, 
                                        eff_size = eff_size, B = B, 
                                        conf_level = conf_level))
    power = np.mean(power_lst)
    return(power)

In [13]:
## Single simulation
single_sim_fun(Nexp = 1000)

0

In [16]:
## Power simulation
power_sim_fun(dat_df=hist_data_df, metric_fun = log_reg_fun, Nexp = int(4e4), 
              eff_size=0.01, Nsim=20)

0.95

In [ ]:
#Alternative parallelized function for higher speed
from joblib import Parallel, delayed
import psutil

def opt_power_sim_fun(dat_df, metric_fun, Nexp, eff_size, Nsim, B = 100, conf_level = 0.9):
    #Parallelized version with joblib
    n_cpu = psutil.cpu_count() #Counting number of cores on machine
    counter = [Nexp] * Nsim
    res_parallel = Parallel(n_jobs = n_cpu)(delayed(single_sim_fun)(Nexp) for Nexp in counter)
    pwr = np.mean(res_parallel)
    return(pwr)
opt_power_sim_fun(dat_df=hist_data_df, metric_fun = log_reg_fun, Nexp = int(1e3), eff_size=0.01, Nsim=10)

## Analyzing and interpreting experimental results

In [5]:
### Logistic regression
log_mod_exp = smf.logit('booked ~ age + gender + oneclick', data = exp_data_df)
res = log_mod_exp.fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.161220
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 booked   No. Observations:                40160
Model:                          Logit   Df Residuals:                    40156
Method:                           MLE   Df Model:                            3
Date:                Sun, 04 Apr 2021   Pseudo R-squ.:                  0.3311
Time:                        14:42:56   Log-Likelihood:                -6474.6
converged:                       True   LL-Null:                       -9679.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         11.6928      0.226     51.819      0.000      11.251      12.135
gender[T.male]     0.2542      0.049      5.182      0.000       0.158       0.350
age               -0.3941      0.006    -61.282      0.000      -0.407      -0.381
oneclick           0.1578      0.047      3.357      0.001       0.066       0.250
==================================================================================
"""

In [6]:
### Calculating average difference in probabilities
def diff_prob_fun(dat_df, reg_model = log_mod_exp):
    
    #Creating new copies of data
    no_button_df = dat_df.loc[:, 'age':'gender']
    no_button_df.loc[:, 'oneclick'] = 0
    button_df = dat_df.loc[:,'age':'gender']
    button_df.loc[:, 'oneclick'] = 1
    
    #Adding the predictions of the model 
    no_button_df.loc[:, 'pred_bkg_rate'] = res.predict(no_button_df)
    button_df.loc[:, 'pred_bkg_rate'] = res.predict(button_df)
    
    diff = button_df.loc[:,'pred_bkg_rate'] \
    - no_button_df.loc[:,'pred_bkg_rate']
    return diff.mean()
    
diff_prob_fun(exp_data_df, reg_model = log_mod_exp)

0.007129714313551981

In [9]:
#Calculating Bootstrap 90%-CI for this difference
boot_CI_fun(exp_data_df, diff_prob_fun, B = 100, conf_level = 0.9)

[0.007039890198715744, 0.007210732728972399]